In [5]:
pip install torch transformers

Note: you may need to restart the kernel to use updated packages.


In [6]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [7]:
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

In [12]:
import pandas as pd

In [13]:
# Load a specific sheet from the Excel file into a DataFrame
df = pd.read_excel('fomc_statements.xlsx', sheet_name='statements')

In [14]:
print(df.columns)

Index(['Date', 'Statement'], dtype='object')


In [15]:
import pandas as pd
import re
from transformers import AutoModelForSequenceClassification, AutoTokenizer

df['Statement'] = df['Statement'].astype(str)
# Text preprocessing
df['Statement'] = df['Statement'].apply(lambda x: x.lower()) # convert text to lowercase
df['Statement'] = df['Statement'].apply(lambda x: re.sub(r'\d+', '', x)) # remove numbers
df['Statement'] = df['Statement'].apply(lambda x: re.sub(r'[^\w\s]', '', x)) # remove punctuation

# Loading tokenizer
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

# Tokenize the statement column
inputs = tokenizer(df['Statement'].tolist(), truncation=True, padding=True, return_tensors="pt")

In [16]:
print(df.head(10))

                  Date                                          Statement
0  2000-02-02 00:00:00  the federal open market committee voted today ...
1  2000-03-21 00:00:00  the federal open market committee voted today ...
2  2000-05-16 00:00:00  the federal open market committee voted today ...
3  2000-06-28 00:00:00  the federal open market committee at its meeti...
4  2000-08-22 00:00:00  the federal open market committee at its meeti...
5  2000-10-03 00:00:00  the federal open market committee at its meeti...
6  2000-11-15 00:00:00  the federal open market committee at its meeti...
7  2000-12-19 00:00:00  the federal open market committee at its meeti...
8  2001-01-03 00:00:00  the federal open market committee decided toda...
9  2001-01-31 00:00:00  the federal open market committee at its meeti...


In [17]:
#dropping  NaN values
df = df.dropna(subset=['Statement'])

In [18]:
df['Statement'] = df['Statement'].apply(lambda x: re.sub(r'<.*?>', '', x))

In [19]:
df['Statement'] = df['Statement'].apply(lambda x: re.sub(r'https?://\S+|www\.\S+', '', x))

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Load the pre-trained FinBERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

# Tokenize the statement column
inputs = tokenizer(df['Statement'].tolist(), truncation=True, padding=True, return_tensors="pt")

# Run the model on the inputs
outputs = model(**inputs)

# Convert the logits to probabilities
probs = torch.nn.functional.softmax(outputs.logits, dim=-1)

# Convert probabilities tensor to numpy array
probs_np = probs.detach().numpy()

# Create a new DataFrame with dates and probabilities
df_results = pd.DataFrame({
    'date': df['date'],
    'negative_prob': probs_np[:, 0],
    'neutral_prob': probs_np[:, 1],
    'positive_prob': probs_np[:, 2]
})
